In [1]:
using JuMP, HiGHS

demand = 800
max_percent = 0.4
suppliers = 1:3
# Piecewise linear functions' parameters
nsegments = 3
slope = [
    9.2 9   7;
    9   8.5 8.3;
    11  8.5 7.5
]
intercept = [
    0 20  420;
    0 25  75;
    0 250 550
]
endpoint = [
    0 100 200 800;
    0 50  250 800;
    0 100 300 800;
]

m = Model(HiGHS.Optimizer)

@variable(m, purchase[suppliers] >= 0)
@variable(m, cost[suppliers]) # t variables in math model
# Using math model names for other piecewise variables
@variable(m, y[suppliers,1:nsegments], Bin)
@variable(m, w[suppliers,1:nsegments])

@objective(m, Min, sum(cost[s] for s in suppliers))

@constraint(m, y_sum_to_1[s in suppliers],
    sum(y[s,i] for i in 1:nsegments) == 1)

@constraint(m, w_sum_to_x[s in suppliers],
    sum(w[s,i] for i in 1:nsegments) == purchase[s])

@constraint(m, w_lower_bound[s in suppliers, i in 1:nsegments],
    endpoint[s,i]y[s,i] <= w[s,i])

@constraint(m, w_upper_bound[s in suppliers, i in 1:nsegments],
    w[s,i] <= endpoint[s,i+1]y[s,i])

@constraint(m, t_defn[s in suppliers],
    cost[s] == 
    sum(slope[s,i]w[s,i] + intercept[s,i]y[s,i] 
                for i in 1:nsegments))

@constraint(m, meet_demand,
    sum(purchase[s] for s in suppliers) == demand)

@constraint(m, max_from_one_suppliers[s in suppliers],
    purchase[s] <= max_percent*demand)

set_silent(m)

optimize!(m)

In [4]:
# Print solution in a different cell (it gets mixed in with solver output otherwise)
println("\nTotal cost: \$", 
    round(objective_value(m),digits=2))
println("Where to buy things from:")
for s in suppliers
    println("Buy ", 
        round(value(purchase[s]),digits=2), 
        " pounds from supplier ", 
        s, " for \$", 
        round(value(cost[s]),digits=2))
    println("Piecewise variables:")
    for i in 1:nsegments
        println("w[", s, ",", i, "] = ", 
            round(value(w[s,i]),digits=2), 
            " y[", s, ",", i, "] = ", 
            round(value(y[s,i]),digits=2))
    end
end


Total cost: $6995.0
Where to buy things from:
Buy 320.0 pounds from supplier 1 for $2660.0
Piecewise variables:
w[1,1] = -0.0 y[1,1] = 0.0
w[1,2] = 0.0 y[1,2] = 0.0
w[1,3] = 320.0 y[1,3] = 1.0
Buy 160.0 pounds from supplier 2 for $1385.0
Piecewise variables:
w[2,1] = -0.0 y[2,1] = 0.0
w[2,2] = 160.0 y[2,2] = 1.0
w[2,3] = 0.0 y[2,3] = 0.0
Buy 320.0 pounds from supplier 3 for $2950.0
Piecewise variables:
w[3,1] = -0.0 y[3,1] = 0.0
w[3,2] = 0.0 y[3,2] = 0.0
w[3,3] = 320.0 y[3,3] = 1.0
